In [11]:
from ads_helper import c,get_incorrectly_tagged_indices,interpret_data
%matplotlib inline

In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
from IPython.display import display
import ipywidgets as widgets
from IPython.core.display import HTML
from time import sleep
import pandas as pd

tdfn =pd.read_csv("known_identity_df.csv",index_col=0)
sdf = pd.read_csv("summary_df.csv",index_col=0)

#using tdfn and sdf from above
classes_of_interest = ['white','black','female','homosexual_gay_or_lesbian','christian','hindu']
options={"All toxic comments being flagged":0,\
         "Non-toxic comments not being flagged as toxic":1,\
         "Percentage of comments that are flagged as toxic are not":2}
inv_options = {v:k for k,v in options.items()}



def display_df_widget(display_df,col):
    display(display_df.sort_values(col,ascending=False))

def sorted_df_widget(display_df,cols):
    return interactive(display_df_widget,display_df = fixed(display_df.loc[classes_of_interest]),col=display_df.columns)


def get_sample_text(base_df,nationality,toxicity):
    toxicity = (toxicity == "Toxic")*1
    sub_df = base_df[(base_df[nationality] > .5) & (base_df["toxicity"] == toxicity)]
    print(sub_df.sample(1)["comment_text"].values[0])

    
def get_explained_text(base_df,nationality,mention_threshold,toxicity,c):
    toxicity = (toxicity == "Toxic")*1
    indices = get_incorrectly_tagged_indices(base_df,nationality,mention_threshold,toxicity)
    #indices = get_incorrect_examples(tdfn,'christian',mention_threshold,0)

    exp = interpret_data(c.predict_proba, base_df.comment_text,base_df["bin_toxicity"],['non_toxic','toxic'],indices,1,True)


def get_recipe_widget(base_df):
    recipe_df = base_df.iloc[:,0:4]
    recipe_df.columns = ["Mentions","Toxic Comments","Non-Toxic Comments","Not Mentioned"]
    return sorted_df_widget(recipe_df,recipe_df.columns)

def get_ingredient_widget(base_df):
    idf = base_df.iloc[:,8:11]
    idf.columns = ["What percentage of the toxic comments are classified as toxic? Low values means many toxic comments were not flagged",\
                   "What percent of comments that are not toxic are flagged as toxic? High values mean many non-toxic comments are flagged as toxic",\
                  "What percentage of comments that are flagged as toxic are not? High values indicate rate of incorrect toxic labeling"]
#     idf.columns =["(# Comments Predicted Toxic)/(# Toxic Comments)",\
#                  "(# Comments Incorrectly Predicted Toxic)/(# Comments Predicted Toxic)"]
    idf = idf.apply(lambda x: round(x,2))
    return sorted_df_widget(idf,idf.columns)

def select_importance_widget():
    pass



def get_sample_text_widget(base_df):
    return interactive(get_sample_text,base_df=fixed(base_df),\
                       nationality = classes_of_interest,\
                       toxicity=["Toxic","Non-Toxic"])


def get_explained_text_widget(base_df):
    return interactive(get_explained_text,base_df = fixed(base_df),\
                                          nationality=classes_of_interest,\
                                          mention_threshold=fixed(.1),
                                          toxicity=["Toxic","Non-Toxic"],\
                                          c=fixed(c))
                              

def get_fpr_risk_widget(base_df):
    idf = base_df.iloc[:,8:11]
    high_f_rate = idf.iloc[:,-1]
        


        
        
def get_question_widget():
    slider = widgets.SelectMultiple(options = options)
    return slider


def get_registered_output(widget,func,*func_args,**kw_func_args):
    output = widgets.Output()
    def update_output(change_dict):
        output.clear_output()
        with output:
            func(change_dict,*func_args,**kw_func_args)
    
    widget.observe(update_output)
    return output



def goal_func(change_dict,base_df):
    option = change_dict['old']['index'][0]
    idf = base_df.iloc[:,8:11].loc[classes_of_interest]
    idf = idf.sort_values(by=[idf.columns[option]],ascending = False)
    idf = idf.iloc[:,[option]]*100
    idf.columns = [inv_options[option]]
    display(HTML(f"<h3>You chose <span style='color:blue'>'{inv_options[option]}'</span> as your goal: </h3>"))
    display(HTML("<p>Accordingly, your groups of interest rank as follows:</p>"))
    if option == 0:
        s = (idf\
         .style\
         .format({idf.columns[0]:'{0:,.0f}%'})\
         .bar(color=['#d65f5f', '#5fba7d'],vmin=0,vmax=100,subset = [idf.columns[0]])\
         .set_caption('Results'))
    else:
        s = (idf\
         .style\
         .format({idf.columns[0]:'{0:,.0f}%'})\
             .bar(subset = [idf.columns[0]], color=['#5fba7d','#d65f5f'],vmin=0,vmax=100)\
         .set_caption('Results'))
    display(s)
        
    
    
    


    

importance_widget = get_question_widget()
goal_output = get_registered_output(importance_widget,goal_func,sdf)
recipe_widget = get_recipe_widget(sdf)
ingredient_widget = get_ingredient_widget(sdf)
text_widget = get_sample_text_widget(tdfn)
exp_text_widget = get_explained_text_widget(tdfn)
display(HTML("<h1>Goal</h1>"))
display(HTML("<h3>I care most about ...</h3>"))
display(importance_widget)
display(HTML("<h1>Recipe</h1>"))
display(recipe_widget)
display(HTML("<h1>Sample Comment</h1>"))
display(text_widget)
display(HTML("<h1>Ingredients</h1>"))
display(ingredient_widget)
display(HTML("<h1>Tagged Comment</h1>"))
display(exp_text_widget)
display(HTML("<h1>Evaluation</h1>"))
display(goal_output)

    

SelectMultiple(options={'All toxic comments being flagged': 0, 'Non-toxic comments not being flagged as toxic'…

interactive(children=(Dropdown(description='col', options=('Mentions', 'Toxic Comments', 'Non-Toxic Comments',…

interactive(children=(Dropdown(description='nationality', options=('white', 'black', 'female', 'homosexual_gay…

interactive(children=(Dropdown(description='col', options=('What percentage of the toxic comments are classifi…

interactive(children=(Dropdown(description='nationality', options=('white', 'black', 'female', 'homosexual_gay…

Output()